<a href="https://colab.research.google.com/github/NguyenVu06/BigDataAnalytics/blob/master/NYCTaxiDataRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load data from the cloud

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://mirror.cc.columbia.edu/pub/software/apache/spark/spark-3.0.0-preview2/spark-3.0.0-preview2-bin-hadoop2.7.tgz
!tar xf spark-3.0.0-preview2-bin-hadoop2.7.tgz
!pip install -q findspark
!rm -rf spark-3.0.0-preview2-bin-hadoop2.7.tgz

tar: spark-3.0.0-preview2-bin-hadoop2.7.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-preview2-bin-hadoop2.7"
import findspark
findspark.init("spark-3.0.0-bin-hadoop3.2")# SPARK_HOME


from pyspark.sql import SparkSession
from pyspark import SparkContext

spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = SparkContext.getOrCreate()

In [ ]:
ls -lah

total 20K
drwxr-xr-x  1 root      root 4.0K Jul 15 22:11 ./
drwxr-xr-x  1 root      root 4.0K Jul 15 22:01 ../
drwxr-xr-x  1 root      root 4.0K Jul 13 16:14 .config/
drwxr-xr-x  1 root      root 4.0K Jul 10 16:29 sample_data/
drwxr-xr-x 13 110302528 1000 4.0K Dec 17  2019 spark-3.0.0-preview2-bin-hadoop2.7/


In [ ]:
!ls -la

total 20
drwxr-xr-x  1 root      root 4096 Jul 15 22:11 .
drwxr-xr-x  1 root      root 4096 Jul 15 22:01 ..
drwxr-xr-x  1 root      root 4096 Jul 13 16:14 .config
drwxr-xr-x  1 root      root 4096 Jul 10 16:29 sample_data
drwxr-xr-x 13 110302528 1000 4096 Dec 17  2019 spark-3.0.0-preview2-bin-hadoop2.7


In [ ]:
!wget https://s3.amazonaws.com/metcs777/taxi-data-sorted-small.csv.bz2

--2020-07-15 22:12:56--  https://s3.amazonaws.com/metcs777/taxi-data-sorted-small.csv.bz2
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.144.206
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.144.206|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 97938609 (93M) [application/x-bzip2]
Saving to: ‘taxi-data-sorted-small.csv.bz2’

taxi-data-sorted-sm 100%[===================>]  93.40M  39.2MB/s    in 2.4s    

2020-07-15 22:12:59 (39.2 MB/s) - ‘taxi-data-sorted-small.csv.bz2’ saved [97938609/97938609]



In [ ]:
#check if spark are running

print(sc.version)


3.0.0-preview2


In [ ]:
lines = sc.textFile("./taxi-data-sorted-small.csv.bz2")

taxilines = lines.map(lambda line: line.split(","))

In [ ]:
#print first row of the data to make sure it is read in correctly
print(taxilines.take(2))

[['07290D3599E7A0D62097A346EFCC1FB5', 'E7750A37CAB07D0DFF0AF7E3573AC141', '2013-01-01 00:00:00', '2013-01-01 00:02:00', '120', '0.44', '-73.956528', '40.716976', '-73.962440', '40.715008', 'CSH', '3.50', '0.50', '0.50', '0.00', '0.00', '4.50'], ['22D70BF00EEB0ADC83BA8177BB861991', '3FF2709163DE7036FCAA4E5A3324E4BF', '2013-01-01 00:02:00', '2013-01-01 00:02:00', '0', '0.00', '0.000000', '0.000000', '0.000000', '0.000000', 'CSH', '27.00', '0.00', '0.50', '0.00', '0.00', '27.50']]


Use pycode from assignment 1 document to clean up the data loaded

In [ ]:
#clean up 
#handling wrong data lines
def isfloat(val):
    try: 
        float(val)
        return True
    except:
        return False
    
#use isFloat to remove lines if they dont have 16 values and a float value !=0 for col 6 and col 12
def correctRows(p):
    if(len(p)==17):
        if(isfloat(p[5]) and isfloat(p[11])):
            if(float(p[5])!=0 and float(p[11])!=0):
                return p
            
#go thru each element per row and keep only the correctRows
cleanTaxiDT = taxilines.filter(correctRows)

### Part1: Top-10 Active Taxis
Many different taxis have had multiple drivers. Write and execute a Spark Python program that computes
the top ten taxis that have had the largest number of drivers. Your output should be a set of (medallion,
number of drivers) pairs


In [ ]:
#set up each row with an 1 index
rdd_1 = cleanTaxiDT.map(lambda x: ((x[0], x[1]), 1))

In [ ]:
rdd_1.first()

(('07290D3599E7A0D62097A346EFCC1FB5', 'E7750A37CAB07D0DFF0AF7E3573AC141'), 1)

In [ ]:
#reduce, aggrgate by the combination of taxi and taxi drivers into a list
rdd_1_reduce = rdd_1.reduceByKey(lambda x, y: x+y)


In [ ]:
# reduce in RDD to count taxi and most frequently used taxi
by_medalionRDD2 = rdd_1_reduce.map(lambda x: (x[0][0],1)).reduceByKey(lambda x, y: x+y)

In [ ]:
#return the top 10 most used taxi by the count
by_medalionRDD2.top(10, lambda x: x[1])

### Part2: Top-10 Best Drivers
We would like to figure out who the top 10 best drivers are in terms of their average earned money per minute spent carrying a customer. The total amount field is the total money earned on a trip. In the end, we
are interested in computing a set of (driver, money per minute) pairs

Create a second clean RDD to remove any rows that contain no value for the minute column and save to cleanTaxiDT2

In [ ]:
def correctRows2(p):
  if(float(p[4]) !=0):
    return p
cleanTaxiDT2 = cleanTaxiDT.filter(correctRows2)

In [ ]:
#get a driver rdd and map a function to get the money per minute value
by_drvr = cleanTaxiDT2.map(lambda x: (x[1], (float(x[11])/(float(x[4])/60), 1.0)))
#aggregate by key=driverID and tke an average using the aggregate byByKey
aggfun = lambda x, y: (x[0]+y[0], x[1]+y[1])
rateRDD = by_drvr.aggregateByKey((0.0,0.0), aggfun, aggfun)

avg_rateRDD = rateRDD.map(lambda x: (x[0], x[1][0]/x[1][1]))


In [ ]:
# Get top 10 drivers
avg_rateRDD.top(10, lambda x: x[1])


[('30B2ACBAF002305533FF0D31D34A7C06', 624.0),
 ('4C3B2A31227663A59E1AA7B45157160D', 520.0),
 ('08026D69508127F4DE855678ABCE7E0A', 300.0),
 ('6E1D7195E38AA7A36B375C1C60AD8632', 240.0),
 ('975D5E840C0F5D9611A71E9A811D11F7', 171.81818181818184),
 ('E8E22AC46DF6AC99C28C6860779A648C', 150.0),
 ('F981706EDD5F7B2BC29A5C9F2D2017C0', 120.4015616285555),
 ('17F72121B9F612D81AC4556CC18D5514', 107.14285714285714),
 ('619BF4020E6542AA0E28FBC7081271F1', 105.88235294117648),
 ('CD9D0B4429613F1B62D1D4FBB72ACF2B', 82.75862068965517)]

### Part3: Best Day to Work
We would like to know which hour of the day is the best time for drivers that has the highest profit per miles. Consider the surcharge amount in dollar for each taxi ride (without tip amount) and the distance in miles, and sum up the rides for each hour of the day (24 hours) – consider the pickup time for your calculation. The profit ratio is the ration surcharge in dollar divided by the travel distance in miles for each specific time
of the day. 

Profit Ratio = (Surcharge Amount in US Dollar) / (Travel Distance in miles)
We are interested to know the time of the day that has the highest profit ratio.

In [ ]:
#col index: surcharge=12, Distance=5, pickup datetime=2

# get the day out of the datetime column
from datetime import datetime

In [ ]:

# create a method to extract hour
def getHour(x):
  dtObj = datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
  return dtObj.strftime('%H')

#create aRDD with the relevant columns: pick up date, surcharge, distance
pt3RDD = cleanTaxiDT.map(lambda x: (getHour(x[2]), (float(x[12]), float(x[5]))))
#RDD to contain sum of all surcharge, all distance for each date as key
byHourRDD = pt3RDD.reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1]))
#RDD to produce date and profit ratio by day
bestDayRDD = byHourRDD.map(lambda x: (x[0], x[1][0]/x[1][1]))



In [ ]:
bestDayRDD.top(10, lambda x: x[1])

[('19', 0.2797969863679002),
 ('18', 0.27278898753108016),
 ('17', 0.24141913629068573),
 ('16', 0.21625007358740558),
 ('20', 0.17366763839394975),
 ('21', 0.16619487745174588),
 ('22', 0.16286985932866763),
 ('23', 0.1540784450680554),
 ('02', 0.15239306867513575),
 ('00', 0.14987512831474073)]

In [ ]:
del byDateRDD


### Part4: Bonus 
How many percent of taxi customers pay with cash and how many percent using electronic cards?
Analyze these payment methods for different time of the day and provide a list of percents for each
day time? 

As a result provide two numbers for total percentages and a list like (hour of day, percent paid card) We would like to measure the efficiency of taxis drivers by finding out their average earned money per mile. (Consider the total amount which includes tips, as their earned money) Implement a Spark job
that can find out the top-10 efficient taxi divers.

What are mean, median, first and third quantiles of tip amount? How do find the median?
Using the IQR outlier detection method find out the top-10 outliers.

In [ ]:
# cash versus credit
# columns: pickup date = 2, payment type = 10

